In [ ]:
import matlab.engine
import numpy as np
import scipy.optimize

# Trial and pd parameters

In [ ]:
# trial and adherence information
drug = "ATV/r"

days = np.double(40) #days to run trial for
increment = 0.01 # days between each timepoint returned
prob_adh = 0.5 if drug != 'TEST' else 0. # probability of taking each pill (independent)
adh_pat = 0 # 0 = random prob for each dose, 1 = random prob each day after missed dose
adh_shuff = np.double(0.) # standard deviation in the time of taking a pill vs scheduled (days)
trial_type = 1 # 1 = suppression trial, 0=maintenance trial
burnin = 7*5 # days before interval sampling begins in maintenance trials
sampling_int = 28 # days between each sampling time in maintenance trials
threshold = 200 # threshold viral load above which failure is declared
mut_frac = 0.2 # threshold fraction of population of mutant above which declared failure via resistance

## Use existing matlab scripts to calculate concentration(t)

In [ ]:
eng = matlab.engine.start_matlab()
eng.cd(r'~/develop/withinhostHIV/MatlabCode/', nargout=0)
eng.eval("addpath('Parameters','Utilities')", nargout=0)

In [ ]:
eng.workspace["drug"] = drug
trial, pd = eng.trial_setup(eng.eval("drug"), days, increment, prob_adh, adh_pat, adh_shuff, trial_type, burnin, sampling_int, threshold, mut_frac, nargout=2)
eng.workspace["trial"] = trial
eng.workspace["pd"] = pd

In [ ]:
dose_t, _ = eng.adh_trajectory(eng.eval("pd.num_doses"), trial, nargout=2)

In [ ]:
c_vec,inhib_vec = eng.drug_trajectory(pd, trial, dose_t, nargout=2)

## Use existing matlab scripts to calculate mutation matrix Q

In [ ]:
smin=0.05; #highest mutant fitness
smax=0.9; #lowest mutant fitness
smiss=0.05; #fitness of strains with missing cost
rfmiss=1; #fold change in resistance for strains with it missing
mfmiss=0; #fractional change in slope for strains with it missing
back_mutation_on = drug != 'TEST' #include(1) or exclude(0) back mutation
direct_multi_hit = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from WT
direct_multi_multi = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from one res strain to another

In [ ]:
mparams,Q = eng.getMutInfo(drug,smin,smax,smiss,rfmiss,mfmiss,back_mutation_on,direct_multi_hit,direct_multi_multi,nargout=2);
eng.workspace["Q"] = Q
Q = eng.eval("Q{:,:};") # get a matrix instead of a matlab table

## Viral dynamics parameters

In [ ]:
R00 = 10.0 # average number of new virions produced by infection with a single virion === average number of new infected cells produced by 1 infected cell
fbp = 0.55 # fraction of blood that is plasma
Vpl = 3*10e3 # volume of plasma in mL
ftcell_pl = 0.02 # fraction of T cells that are circulating in blood (vs in lymph system)
hl_lr = 44*30.5 # half life of decay of latent reservoir, days
A = 100 # total reactivation of latent cells per day
flr = 1e-6# fraction of CD4 T cells that are latently infected at equilibrium

scale_cd4_body = (Vpl*10**3)/(fbp*ftcell_pl) # factor to go from T cell per ul blood to whole body

fa = 0.01 # fraction of infected CD4 cells that support productive vs abortive infection
dx = 0.05 # death rate of uninfected cells (per day) and by assumption dx == d, rate of death without viral cytolytic effects
L = scale_cd4_body*1000*dx # uninfected cells produced per day (/ul)
a = A/(flr*L/dx) # rate of exit of latent cells (per day)
dz = np.log(2)/(hl_lr)-a # death rate of latently infected cells (per day)

dy = 1 # TOTAL death rate of infected cells (per day) (=death due to burst (k) + death without viral cytolytic effects)
k = dy-dx # rate of death+emission of a burst of virions (per day)
# CHECK SCI NOTO
N = 2.38e5
assert(N > R00)

# probability of a single virion establishing infection, solved implicitly
p_est_solution = scipy.optimize.least_squares(lambda p_est: R00*(1-(1-p_est)**N) - N*p_est, 0.1)
assert(p_est_solution.success)
p_est = p_est_solution.x

dv = 25 # TOTAL death rate of virions (per day)
beta = R00 * dy * dv * dx / (k * N * fa * L) # infectivity rate (per day*infectious-target cell pair)
c = dv-beta*L/dx # clearance rate of virions (per day);
assert(c > 0)
g = flr*dy/dx*(a+dz)/(fa*(1-1/R00)) # fraction of new infections that become latent

In [ ]:
# Access matlab parameters
eng.workspace["mparams"] = mparams
IC50 = eng.eval('pd.IC50')
m = eng.eval('pd.m')
cost = eng.eval('mparams.cost')
rf = eng.eval('mparams.rf')
mf = eng.eval('mparams.mf')
t_vec = eng.eval('(0:trial.increment:trial.days);')

## Check beta(t) (from matlab) against beta(t) (from python)

In [ ]:
beta_t, beta_u_t = eng.calculate_beta_of_t(t_vec, beta, c_vec, IC50, m, cost, rf, mf, eng.eval('height(mparams)'), nargout=2)

In [ ]:
from numba import jit

@jit(nopython=True)
def calculate_beta_t(beta_0, concentration, IC50, m, cost, rf, mf):
    # B/[cost * (1 + concentration)/(IC50*rf)^(m*(1+mf))]
    B = beta_0
    denominator = cost * (1 + (concentration/(IC50*rf))**(m*(1+mf)))
    return B/denominator


# make sure we didn't mess up our function definition
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T), np.asarray(beta_u_t)))
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1), np.asarray(beta_t)))

beta_u_t = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T)
beta_t   = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1)

beta_t = np.concatenate([beta_t, beta_u_t], axis=1)
t_vec = np.asarray(t_vec).squeeze()
dose_t = np.asarray(dose_t).T.squeeze()

In [ ]:
sampled_concentration = np.asarray(c_vec)
# closed around parameters
# pulls the nearest beta pre-calculated to the left of the current time

increment = eng.eval('trial.increment')

@jit(nopython=True)
def discrete_beta_of_t(t):
    # last time where beta was precalculated *before* t
    # specifically want *before* t so that we don't look ahead while integrating past a discontinuity
    t_discrete = int(np.floor(t/increment))
    return beta_t[t_discrete]

Q = np.asarray(Q)
@jit(nopython=True)
def force_of_infection(t):
    beta = discrete_beta_of_t(t)
    # calculate force of infection and flatten in row major order, which will make the matrix
    # read correctly in our 1-dimensional vector of rates

    # **NOTE* we moved FA to the stoichiometry matrix, otherwise it would be a factor below!
    return (np.expand_dims(beta,1) * Q).flatten()

TODO: triple check to make sure this is the right orientation of beta_i relative to Q

In [ ]:
Q

In [ ]:
print(Q[0,1], Q[1,0], ". Q[0,1] > Q[1,0]:", Q[0,1] > Q[1,0])

In [ ]:
row_i, col_i = np.triu_indices_from(Q)
for i,j in zip(list(row_i), list(col_i)):
    print(Q[i,j] > Q[j,i])

In [ ]:
np.expand_dims(beta_t[0],1)

# Define model

In [ ]:
import reactionmodel.reactionmodel as reactionmodel
from importlib import reload
reload(reactionmodel)
num_strains = beta_t.shape[1]
num_strains

In [ ]:
ys = [reactionmodel.Species(f'infected cells strain {i}', f'y{i}') for i in range(num_strains)]
zs = [reactionmodel.Species(f'latent cells strain {i}', f'z{i}') for i in range(num_strains)]
vs = [reactionmodel.Species(f'viruses strain {i}', f'v{i}') for i in range(num_strains)]
x = reactionmodel.Species("target cells", "x")
all_species = [x] + ys + vs + zs

In [ ]:
target_cell_birth = reactionmodel.Reaction("target cell birth", [], [x], k=L)
target_cell_death = reactionmodel.Reaction("target cell death", [x], [], k=dx)
infected_death = [reactionmodel.Reaction(f"death of infected cell strain {i}", [ys[i]], [], k=(dy-k)) for i in range(num_strains)]
infected_cell_burst = [reactionmodel.Reaction(f"burst of infected cell strain {i}", [ys[i]], [(vs[i], N)], k=k) for i in range(num_strains)]
into_latency = [reactionmodel.Reaction(f"strain {i} --> latency", [ys[i]], [zs[i]], k=g) for i in range(num_strains)]
out_of_latency = [reactionmodel.Reaction(f"latent --> strain {i}", [zs[i]], [ys[i]], k=a) for i in range(num_strains)]
death_of_latent = [reactionmodel.Reaction(f"latent {i} death", [zs[i]], [], k=dz) for i in range(num_strains)]
viral_clearance = [reactionmodel.Reaction(f"clearance of strain {i} virion", [vs[i]], [], k=c) for i in range(num_strains)]

infections = []
for i in range(num_strains):
    for j in range(num_strains):
        infections.append(reactionmodel.Reaction(f"infection of x by {i}->{j}", [x, vs[i]], [(ys[j], fa)]))
infection_family = reactionmodel.ReactionRateFamily(infections, k=force_of_infection)

all_reactions = [target_cell_birth, target_cell_death] + infected_death + infected_cell_burst + into_latency + out_of_latency + death_of_latent + viral_clearance + [infection_family]


In [ ]:
from types import FunctionType as function
from numba.core.registry import CPUDispatcher
isinstance(all_reactions[-1].k, CPUDispatcher)

In [ ]:
model = reactionmodel.Model(all_species, all_reactions, jit=True)

In [ ]:
for reaction in model.all_reactions:
    print(reaction)

# Run forward simulation

In [ ]:
import hybrid

In [ ]:
rng = np.random.default_rng()

extra_discontinuities = list(np.linspace(0.0,40.0,400))
extra_discontinuities = []

y0 = np.zeros(len(all_species))
y0[model.species_index[x]] = L/dx/R00
y0[model.species_index[ys[0]]] = fa*(1-1/R00)*L/dy
y0[model.species_index[zs[0]]] = y0[model.species_index[ys[0]]]*g/(a+dz)

In [ ]:
%load_ext jupyterflame

In [ ]:
rng = np.random.default_rng()
result = hybrid.forward_time(
    y0,
    [0, 40.0],
    lambda p: hybrid.partition_by_threshold(p, 200),
    model.k_jit,
    model.stoichiometry(),
    model.rate_involvement(),
    rng,
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True
)

In [ ]:
result

In [ ]:
scale_cd4_body=(Vpl*10**3)/(fbp*ftcell_pl)
#scale_cd4_virus=kdv*2*1e3/fbp

In [ ]:
mnames = list(np.asarray(eng.eval('mparams.Properties.RowNames')))

import matplotlib.pyplot as plt
plt.figure()
plt.plot(result.t_history, result.y_history[0].T/scale_cd4_body, label='x')
plt.yscale('linear')
plt.ylabel('uninfected CD4 cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([0, 1025])

plt.figure()
plt.plot(result.t_history, result.y_history[1:1+len(ys)].T/scale_cd4_body, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('infected cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-9, 1e3])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(vs[0]):model.species.index(vs[-1])+1].T/Vpl/1000, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('virions (copies/mL plasma)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-3, 1e7])

plt.figure()
plt.plot(t_vec, beta_t*R00/beta, label=['wildtype']+mnames)
plt.legend()
plt.ylabel('R_u')
plt.xlabel('t')
plt.yscale('linear')

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1].T*(10**6)/(L/dx), label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('LR size(per 10^6 baseline CD4)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 10e2])

In [ ]:
result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1][:,-1]/(10**6)/(L/dx)

In [ ]:
print(model.pretty(skip_blanks=True))

In [ ]:
model.k(0)[-11] * L/dx * N

In [ ]:
def dydt_factory(calculate_propensities):
    def dydt(t, y_expanded, k, deterministic_mask, stochastic_mask, hitting_point):
        y = y_expanded[:-1]
        k = k(t)
        propensities = calculate_propensities(y, k)
        deterministic_propensities = propensities * deterministic_mask
    
        deterministic_propensities
        dydt = np.zeros(len(y_expanded))
    
        ## reaction rates used multiple times 
        _infections = deterministic_propensities[r.index(viral_clearance[-1])+1:len(r)].reshape(n, n)
        new_infections_by_strain = _infections.sum(axis=0)
        loss_of_virions_by_strain = _infections.sum(axis=1)
    
        ic_death = deterministic_propensities[r.index(infected_death[0]):r.index(infected_death[0])+n]
        ic_burst = deterministic_propensities[r.index(infected_cell_burst[0]):r.index(infected_cell_burst[0])+n]
        to_latent = deterministic_propensities[r.index(into_latency[0]):r.index(into_latency[0])+n]
        from_latent = deterministic_propensities[r.index(out_of_latency[0]):r.index(out_of_latency[0])+n]
    
        ## Infected cells
        dydt[1:1+n] = (
            fa * new_infections_by_strain
            + from_latent
            - ic_death
            - ic_burst
            - to_latent
        )
    
        ## Virions
        dydt[1+n:1+2*n] = (
            burst_size * ic_burst
            - loss_of_virions_by_strain
            - deterministic_propensities[r.index(viral_clearance[0]):r.index(viral_clearance[0])+n]
        )
    
        ## Latent cells
        dydt[species.index(zs[0]):species.index(zs[0])+n] = (
            to_latent
            - from_latent
            - deterministic_propensities[r.index(death_of_latent[0]):r.index(death_of_latent[0])+n]
        )
    
        ## Target cells
        dydt[species.index(x)] = deterministic_propensities[r.index(target_cell_birth)] - deterministic_propensities[r.index(target_cell_death)] - np.sum(loss_of_virions_by_strain)
        dydt[-1] = np.sum(propensities * stochastic_mask)
        return dydt
    return dydt

In [ ]:
#rates = N @ deterministic_propensities

#fa = 0.01
species = tuple(model.species.copy())
r = tuple(model.reactions.copy())
n = len(ys)
burst_size = N

num_pathways = len(r)
clearance_start = r.index(viral_clearance[0])
clearance_end = r.index(viral_clearance[-1])
ic_death_start = r.index(infected_death[0])
ic_burst_start = r.index(infected_cell_burst[0])
to_latent_start = r.index(into_latency[0])
from_latent_start = r.index(out_of_latency[0])
latent_death_start = r.index(death_of_latent[0])
target_birth = r.index(target_cell_birth)
target_death = r.index(target_cell_death)

latent_start = species.index(zs[0])
ic_start = species.index(ys[0])
virus_start = species.index(vs[0])
x_start = species.index(x)

def jit_dydt_factory(calculate_propensities):
    @jit(nopython=True)
    def dydt(t, y_expanded, k, deterministic_mask, stochastic_mask, hitting_point):
        y = y_expanded[:-1]
        k = k(t)
        propensities = calculate_propensities(y, k)
        deterministic_propensities = propensities * deterministic_mask
    
        deterministic_propensities
        dydt = np.zeros(len(y_expanded))
    
        ## reaction rates used multiple times 
        _infections = deterministic_propensities[clearance_end+1:clearance_end+1+n**2].reshape(n, n)
        new_infections_by_strain = _infections.sum(axis=0)
        loss_of_virions_by_strain = _infections.sum(axis=1)
    
        ic_death = deterministic_propensities[ic_death_start:ic_death_start+n]
        ic_burst = deterministic_propensities[ic_burst_start:ic_burst_start+n]
        to_latent = deterministic_propensities[to_latent_start:to_latent_start+n]
        from_latent = deterministic_propensities[from_latent_start:from_latent_start+n]
    
        ## Infected cells
        dydt[ic_start:ic_start+n] = (
            fa * new_infections_by_strain
            + from_latent
            - ic_death
            - ic_burst
            - to_latent
        )
    
        ## Virions
        dydt[virus_start:virus_start+n] = (
            burst_size * ic_burst
            - loss_of_virions_by_strain
            - deterministic_propensities[clearance_start:clearance_start+n]
        )
    
        ## Latent cells
        dydt[latent_start:latent_start+n] = (
            to_latent
            - from_latent
            - deterministic_propensities[latent_death_start:latent_death_start+n]
        )
    
        ## Target cells
        dydt[x_start] = (
            deterministic_propensities[target_birth]
            - deterministic_propensities[target_death]
            - np.sum(loss_of_virions_by_strain)
        )
        
        dydt[-1] = np.sum(propensities * stochastic_mask)
        return dydt
    return dydt

In [ ]:
%%flame
rng = np.random.default_rng(0)
result = hybrid.forward_time(
    y0,
    [0, 40.0],
    lambda p: hybrid.partition_by_threshold(p, 100),
    model.k_jit,
    model.stoichiometry(),
    model.rate_involvement(),
    rng,
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True,
    expert_dydt_factory = jit_dydt_factory
)

In [ ]:
%%time
rng = np.random.default_rng()
result = hybrid.forward_time(
    y0,
    [0, 100.0],
    lambda p: hybrid.partition_by_threshold(p, 100),
    model.k_jit,
    model.stoichiometry(),
    model.rate_involvement(),
    rng,
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True,
    expert_dydt_factory = jit_dydt_factory
)

In [ ]:
result

In [ ]:
mnames = list(np.asarray(eng.eval('mparams.Properties.RowNames')))

import matplotlib.pyplot as plt
plt.figure()
plt.plot(result.t_history, result.y_history[0].T/scale_cd4_body, label='x')
plt.yscale('linear')
plt.ylabel('uninfected CD4 cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([0, 1025])

plt.figure()
plt.plot(result.t_history, result.y_history[1:1+len(ys)].T/scale_cd4_body, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('infected cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-9, 1e3])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(vs[0]):model.species.index(vs[-1])+1].T/Vpl/1000, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('virions (copies/mL plasma)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-3, 1e7])

plt.figure()
plt.plot(t_vec, beta_t*R00/beta, label=['wildtype']+mnames)
plt.legend()
plt.ylabel('R_u')
plt.xlabel('t')
plt.yscale('linear')

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1].T*(10**6)/(L/dx), label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('LR size(per 10^6 baseline CD4)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 10e2])